# Imports and setting up viz

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

#import personnal tools
import sys
sys.path.append('../../python_tools/')
from tools import *
from tools_routing import *

# Load files

Looking at simulations over 30 years (1979-2008) to better understand the stabilisation dynamics of the routing (espacially native with MERIT grid)

## Standard (nat7)

In [ ]:
#open nat7/nat7_sechiba_history_all.nc as std
filename = 'nat7/nat7_sechiba_history_all.nc'
std = xr.open_dataset(filename)
std

## Native routing topo 0.5° (nat8)

In [ ]:
#open nat8/nat8_sechiba_history_all.nc as natORC_half
filename = 'nat8/nat8_sechiba_history_all.nc'
natORC_half = xr.open_dataset(filename)
natORC_half

In [ ]:
#open nat8/diag_routing_monthly_all.nc as natDIAG_half
filename = 'nat8/nat8_diag_routing_monthly_all.nc'
natDIAG_half = xr.open_dataset(filename)
natDIAG_half

In [ ]:
#open nat8/nat8_diag_routing_r_all.nc as natDIAG__R_half
filename = 'nat8/nat8_diag_routing_r_all.nc'
natDIAG_R_half = xr.open_dataset(filename)
natDIAG_R_half

## Native routing topo MERIT (nat10)

In [ ]:
#open sechiba_history_all.nc as natORC
filename = 'nat10/dt3600/nat10_dt3600_sechiba_history_all.nc'
natORC = xr.open_dataset(filename)
natORC

In [ ]:
#open diag_routing_all.nc
filename = 'nat10/dt3600/nat10_dt3600_diag_routing_all.nc'
natDIAG = xr.open_dataset(filename)
natDIAG

In [ ]:
#open file with hydrographs on 30 years sim
filename = 'nat10/dt3600/nat10_dt3600_diag_routing_r_all.nc'
natDIAG_R = xr.open_dataset(filename)
natDIAG_R

# File manipulations

In [ ]:
#Change dataset names attribute for automatic labeling
natORC.attrs['name'] = 'sechiba_history_native'
natORC_half.attrs['name'] = 'sechiba_history_native_half'
natDIAG_half.attrs['name'] = 'diag_routing_monthly_native_half'
natDIAG_R_half.attrs['name'] = 'diag_routing_r_native_half'

In [ ]:
#rename time_counter to time in all datasets
std = std.rename({'time_counter' : 'time'})
natORC_half = natORC_half.rename({'time_counter' : 'time'})
natDIAG_half = natDIAG_half.rename({'time_counter' : 'time'})
natDIAG_R_half = natDIAG_R_half.rename({'time_counter' : 'time'})
natDIAG = natDIAG.rename({'time_counter' : 'time'})
natDIAG_R = natDIAG_R.rename({'time_counter' : 'time'})
natORC = natORC.rename({'time_counter' : 'time'})

In [ ]:
#Rename variables in natDIAG_R_month and natDIAG_R
dict = {'routing_stream_reservoir_r' : 'streamr',
        'routing_fast_reservoir_r' : 'fastr',
        'routing_slow_reservoir_r' : 'slowr',
        'routing_hydrographs_r' : 'hydrographs'}
#change the variables according to the dictionary
natDIAG_R = natDIAG_R.rename(dict)
natDIAG_R_half = natDIAG_R_half.rename(dict)

In [ ]:
#Rename lat_domain_landpoints to lat and lon_domain_landpoints to lon in natDIAG
# natDIAG = natDIAG.rename({'lat_domain_landpoints':'lat','lon_domain_landpoints':'lon'})

#OR do it for lat_domain_landpoints_routing and lon_domain_landpoints_routing (for reservoirs)

# natDIAG = natDIAG.rename({'lat_domain_landpoints_routing':'lat','lon_domain_landpoints_routing':'lon'})
# natDIAG_half = natDIAG_half.rename({'lat_domain_landpoints_routing':'lat','lon_domain_landpoints_routing':'lon'})

In [ ]:
#Rename variables in natDIAG_month
dict = {'routing_stream_reservoir' : 'streamr',
        'routing_fast_reservoir' : 'fastr',
        'routing_slow_reservoir' : 'slowr',
        'routing_drainage' : 'drainage',
        'routing_runoff' : 'runoff',
        'routing_riverflow' : 'riverflow',
        'routing_coastalflow' : 'coastalflow'}
#change the variables that exist according to the dictionary
natDIAG = natDIAG.rename(dict)
natDIAG_half = natDIAG_half.rename(dict)

# Maps

In [ ]:
#Display map of var for ds
var='fastr'
ds = natORC
plotvar = ds[var].mean(dim='time')
fig = plt.figure(figsize=(10, 6))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
#use colormap with 20 bins from min to max value
plotvar.plot(ax=ax, transform=ccrs.PlateCarree(), cmap=reds)
plt.title(var + ' (' + ds.name + ')')

In [ ]:
# Display map of var
var = 'streamr'

ds_std = std
plotvar_std = ds_std[var].mean(dim='time')

ds_nat = natORC
plotvar_nat = ds_nat[var].mean(dim='time')

# Create a figure with two subplots side by side
fig, axs = plt.subplots(1, 2, figsize=(16, 6), subplot_kw={'projection': ccrs.PlateCarree()})
fig.suptitle(var + ' average over the period')

# Plot for std dataset
axs[0].coastlines()
plotvar_std.plot(ax=axs[0], transform=ccrs.PlateCarree(), cmap=reds)
axs[0].set_title(f'{var} ({ds_std.name})')

# Plot for natORC dataset
axs[1].coastlines()
plotvar_nat.plot(ax=axs[1], transform=ccrs.PlateCarree(), cmap=reds)
axs[1].set_title(f'{var} ({ds_nat.name})')

# Time series

In [ ]:
river_coordinates = {
    'Ebre': {'lon': 0.75, 'lat': 40.75},
    'Minho': {'lon': -8.75, 'lat': 42.25},
    'Douro': {'lon': -8.75, 'lat': 41.25},
    'Tage': {'lon': -9.25, 'lat': 39.25},
    'Guadiana': {'lon': -7.25, 'lat': 38.25},
    'Guadalquivir': {'lon': -6.25, 'lat': 36.75}
}

In [ ]:
river_coordinates_native = {
    'Ebre': {'lon': 0.86, 'lat': 40.725},
    'Minho': {'lon': -8.875, 'lat': 41.86},
    'Douro': {'lon': -8.675, 'lat': 41.1375},
    'Tage': {'lon': -9.175, 'lat': 38.6875},
    'Guadiana': {'lon': -7.39, 'lat': 37.175},
    'Guadalquivir': {'lon': -6.34, 'lat': 36.79}
}

In [ ]:
#Time series on a specific lon-lat point
year_min=1979
year_max=2008

# lon=-6.34
# lat=36.79
lon=-8.75
lat=39.25

var="streamr"
ds1=natORC
ds1 = ds1.where(ds1['time.year'] >= year_min, drop=True).where(ds1['time.year'] <= year_max, drop=True)
ds2=natORC_half
ds2 = ds2.where(ds2['time.year'] >= year_min, drop=True).where(ds2['time.year'] <= year_max, drop=True)
ds3=std
ds3 = ds3.where(ds3['time.year'] >= year_min, drop=True).where(ds3['time.year'] <= year_max, drop=True)
#plot the time series of hydrographs at the point (lon,lat)
fig = plt.figure(figsize=(10, 6))
ax = plt.axes()
ax.grid()
ds2[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds2.name)
ds3[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds3.name)
ds1[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds1.name)
plt.title(var + ' at (' + str(lon) + ',' + str(lat) + ')')
plt.legend()

In [ ]:
#Time series on grid average
year_min=1979
year_max=2008
var="streamr"
ds1= natORC
ds1 = ds1.where(ds1['time.year'] >= year_min, drop=True).where(ds1['time.year'] <= year_max, drop=True)
ds2=natORC_half
ds2 = ds2.where(ds2['time.year'] >= year_min, drop=True).where(ds2['time.year'] <= year_max, drop=True)
ds3=std
ds3 = ds3.where(ds3['time.year'] >= year_min, drop=True).where(ds3['time.year'] <= year_max, drop=True)
#plot the time series of var
fig = plt.figure(figsize=(8, 5))
ax = plt.axes()
ax.grid()
ds2[var].mean(dim=('lat','lon')).plot(ax=ax, label=ds2.name)
ds3[var].mean(dim=('lat','lon')).plot(ax=ax, label=ds3.name)
ds1[var].mean(dim=('lat','lon')).plot(ax=ax, label=ds1.name)
plt.title(var + ' grid average (' + std[var].units + ')')
plt.legend()

In [ ]:
#cycle saisonnier pour 6 points de grille
var = "hydrographs"
ds1 = std
ds2 = natDIAG_R_half
ds3 = natDIAG_R
spinup_last_year=1994
#remove years before spinup for all datasets
ds1 = ds1.where(ds1['time.year'] >= spinup_last_year, drop=True)
ds2 = ds2.where(ds2['time.year'] >= spinup_last_year, drop=True)
ds3 = ds3.where(ds3['time.year'] >= spinup_last_year, drop=True)

# Create subplots for each river in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over rivers and plot hydrographs
for i, (river, coordinates) in enumerate(river_coordinates.items()):
    lon = coordinates['lon']
    lat = coordinates['lat']
    
    ax = axes[i]

    #Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
    # ds1[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds1.name)
    # ds2[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds2.name)

for i, (river, coordinates) in enumerate(river_coordinates_native.items()):
    lon = coordinates['lon']
    lat = coordinates['lat']
    
    ax = axes[i]

    # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
    ds3[var].sel(lon=lon, lat=lat, method='nearest').groupby('time.month').mean(dim='time').plot(ax=ax, label=ds3.name)
    
    ax.legend()
    #set subplot title to river name
    ax.set_title(river)

# Remove any remaining empty subplots
for i in range(len(river_coordinates), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle("Cycle saisonnier du débit à l'exutoire (m³/s, à partir de {})".format(spinup_last_year))
plt.tight_layout()

In [ ]:
#série temporelle pour 6 points de grille
var = "hydrographs"
ds1 = std
ds2 = natDIAG_R_half
ds3 = natDIAG_R

# Create subplots for each river in a 2x3 grid
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(20, 10))

# Flatten the axes array for easier iteration
axes = axes.flatten()

# Iterate over rivers and plot hydrographs
for i, (river, coordinates) in enumerate(river_coordinates.items()):
    lon = coordinates['lon']
    lat = coordinates['lat']
    
    ax = axes[i]

    # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
    ds1[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds1.name)
    ds2[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds2.name)

for i, (river, coordinates) in enumerate(river_coordinates_native.items()):
    lon = coordinates['lon']
    lat = coordinates['lat']
    
    ax = axes[i]

    # Compute the annual cycle of hydrographs at the point (lon, lat) for each dataset
    ds3[var].sel(lon=lon, lat=lat, method='nearest').plot(ax=ax, label=ds3.name)
    
    ax.legend()
    #set subplot title to river name
    ax.set_title(river)

# Remove any remaining empty subplots
for i in range(len(river_coordinates), len(axes)):
    fig.delaxes(axes[i])

plt.suptitle("Série temporelle du débit à l'exutoire (m³/s)")
plt.tight_layout()